In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.preprocessing import normalize
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
from tqdm import tqdm
from matplotlib import pyplot as plt
seed = 1234
torch.manual_seed(seed)

# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime
# Clear any logs from previous runs
!rm -rf ./logs/ 

2023-10-13 20:34:12.911354: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-13 20:34:12.959770: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-13 20:34:12.961606: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-13 20:34:13.847043: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Import données

In [2]:
(x_train, y_train), (x_test, y_test) =tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=1234
)


# Description dataset

CRIM     per capita crime rate by town

 ZN       proportion of residential land zoned for lots over 25,000 sq.ft.

 INDUS    proportion of non-retail business acres per town

 CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)

 NOX      nitric oxides concentration (parts per 10 million)

 RM       average number of rooms per dwelling

 AGE      proportion of owner-occupied units built prior to 1940

 DIS      weighted distances to five Boston employment centres

 RAD      index of accessibility to radial highways

 TAX      full-value property-tax rate per $10,000

 PTRATIO  pupil-teacher ratio by town

 B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town

 LSTAT    % lower status of the population
 
 MEDV     Median value of owner-occupied homes in $1000's

In [3]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
data = np.hstack((x_train, y_train.reshape(-1,1)))
data = pd.DataFrame(data, columns=column_names)
data.head()
data.describe()

(404, 13)
(404,)
(102, 13)
(102,)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000
mean,3.742918,11.877475,11.200569,0.069307,0.556849,6.299928,69.308168,3.788781,9.626238,410.185644,18.460149,355.311782,12.777054,22.426733
std,8.385410,23.950926,6.908259,0.254290,0.116526,0.702557,28.004850,2.152334,8.751116,169.116234,2.180405,93.327775,7.156010,9.174751
min,0.006320,0.000000,0.740000,0.000000,0.385000,3.863000,6.000000,1.174200,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082125,0.000000,5.190000,0.000000,0.453000,5.886500,45.550000,2.096725,4.000000,279.000000,17.400000,375.300000,7.177500,16.575000
50%,0.260420,0.000000,9.795000,0.000000,0.538000,6.211500,79.450000,3.100900,5.000000,335.000000,19.100000,391.305000,11.490000,21.150000
75%,3.681942,12.500000,18.100000,0.000000,0.631000,6.626000,94.175000,5.035675,24.000000,666.000000,20.200000,396.080000,17.102500,25.000000
max,73.534100,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,36.980000,50.000000


# Normalisation

In [4]:
class Dat(Dataset):
	def __init__(self, x, y):
		super(Dat, self).__init__()
		self.labels = torch.from_numpy(y).double()
		self.data = torch.from_numpy(x).double()
	def __getitem__(self, index):
		return self.data[index], self.labels[index]
	def __len__(self):
		return len(self.labels)

In [5]:
y_train /= y_train.max()
x_train = normalize(x_train, axis=0)
x_test = normalize(x_test, axis=0)
y_test /= y_test.max()
data_test = Dat(x_test, y_test)
x_test2 = data_test[0][0]
y_test2 = data_test[1][0]
train_dataset = Dat(x_train, y_train)

In [ ]:
writer = SummaryWriter(comment='mini-batch-size-40')
model = torch.nn.Linear(13, 1).double()
lossfn = torch.nn.MSELoss()
trainloader = DataLoader(dataset=train_dataset, batch_size=40, shuffle=True)
learningRate = 0.0005
epochs = 1000

for e in tqdm(range(epochs)):
    loss_mean = [] # pour calculer ensuite la moyenne des loss sur tous les batches
    loss_mean_test = [] # pour calculer ensuite la moyenne des loss sur le test

    for x_batch, y_batch in trainloader: # batch
        # forward
        y_pred = model(x_batch)
        loss = lossfn(y_pred, y_batch)
        writer.add_scalar("Loss/train", loss, e)
        #backward
        loss.backward()
        #descente de gradient
        with torch.no_grad():
            model.weight -= learningRate * model.weight.grad
            model.bias -= learningRate * model.bias.grad
        model.weight.grad.zero_() #remise a zero des gradients
        model.bias.grad.zero_()
    with torch.no_grad():
        y_pred_test = model(x_test2)
        loss_test = lossfn(y_pred_test, y_test2)
        loss_mean_test.append(loss_test.item())
        writer.add_scalar("Loss/test", loss_test, e)

    if (e % 10) == 0:
        print("epoch %d " % e , "train MSE: ", np.array(loss_mean).mean(), "val MSE: ", np.array(loss_mean_test).mean())
        

  0%|                                                                                          | 0/1000 [00:00<?, ?it/s]/home/pidoux/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([40])) that is different to the input size (torch.Size([40, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/pidoux/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/pidoux/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([13])) that is different to the input size (torch.

epoch 0  train MSE:  nan val MSE:  0.013653188995687546
epoch 10  train MSE:  nan val MSE:  0.021326172316493
epoch 20  train MSE:  nan val MSE:  0.02983027694441851


  4%|███▌                                                                             | 44/1000 [00:01<00:16, 56.43it/s]

epoch 30  train MSE:  nan val MSE:  0.038789799623010005
epoch 40  train MSE:  nan val MSE:  0.047734227211606334
epoch 50  train MSE:  nan val MSE:  0.05655666574906647


  7%|█████▉                                                                           | 74/1000 [00:01<00:11, 78.97it/s]

epoch 60  train MSE:  nan val MSE:  0.06495340622812748
epoch 70  train MSE:  nan val MSE:  0.07299981073835049


  9%|███████▌                                                                         | 94/1000 [00:01<00:10, 84.55it/s]

epoch 80  train MSE:  nan val MSE:  0.08067489079872593
epoch 90  train MSE:  nan val MSE:  0.08794543735789859


 10%|████████▎                                                                       | 104/1000 [00:01<00:10, 84.60it/s]

In [ ]:
writer = SummaryWriter(comment='batch')
model = torch.nn.Linear(13, 1).double()
lossfn = torch.nn.MSELoss()
trainloader = DataLoader(dataset=train_dataset, batch_size=len(x_train), shuffle=True)
learningRate = 0.001
epochs = 1000

for e in tqdm(range(epochs)):
    loss_mean = [] # pour calculer ensuite la moyenne des loss sur tous les batches
    loss_mean_test = [] # pour calculer ensuite la moyenne des loss sur le test

    for x_batch, y_batch in trainloader: # batch
        # forward
        mult = model(x_batch)
        loss = lossfn(mult, y_batch)
        loss_mean.append(loss.item())
        writer.add_scalar("Loss/train", loss, e)
        #backward
        loss.backward()
        #descente de gradient
        with torch.no_grad():
            model.weight -= learningRate * model.weight.grad
            model.bias -= learningRate * model.bias.grad
        model.weight.grad.zero_() #remise a zero des gradients
        model.bias.grad.zero_()
    with torch.no_grad():
        mult_test = model(x_test2)
        loss_test = lossfn(mult_test, y_test2)
        loss_mean_test.append(loss_test.item())
        writer.add_scalar("Loss/test", loss_test, e)

    if (e % 10) == 0:
        print("epoch %d " % e , "train MSE: ", np.array(loss_mean).mean(), "val MSE: ", np.array(loss_mean_test).mean())
        

In [ ]:
writer = SummaryWriter(comment='stochastique')
model = torch.nn.Linear(13, 1).double()
lossfn = torch.nn.MSELoss()
trainloader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)
learningRate = 0.001
epochs = 1000

for e in tqdm(range(epochs)):
    loss_mean = [] # pour calculer ensuite la moyenne des loss sur tous les batches
    loss_mean_test = [] # pour calculer ensuite la moyenne des loss sur le test

    for x_batch, y_batch in trainloader: # batch
        # forward
        mult = model(x_batch)
        loss = lossfn(mult, y_batch)
        loss_mean.append(loss.item())
        writer.add_scalar("Loss/train", loss, e)
        #backward
        loss.backward()
        #descente de gradient
        with torch.no_grad():
            model.weight -= learningRate * model.weight.grad
            model.bias -= learningRate * model.bias.grad
        model.weight.grad.zero_() #remise a zero des gradients
        model.bias.grad.zero_()
    with torch.no_grad():
        mult_test = model(x_test2)
        loss_test = lossfn(mult_test, y_test2)
        loss_mean_test.append(loss_test.item())
        writer.add_scalar("Loss/test", loss_test, e)

    if (e % 10) == 0:
        print("epoch %d " % e , "train MSE: ", np.array(loss_mean).mean(), "val MSE: ", np.array(loss_mean_test).mean())

In [ ]:
writer = SummaryWriter(comment='optim')
model = torch.nn.Linear(13, 1).double()
lossfn = torch.nn.MSELoss()
trainloader = DataLoader(dataset=train_dataset, batch_size=20, shuffle=True)
learningRate = 0.001
epochs = 1000

optim=torch.optim.SGD(params=model.parameters(), lr=learningRate)
optim.zero_grad()
for e in tqdm(range(epochs)): 
    for x_batch, y_batch in trainloader:
        loss = lossfn(model(x_batch),y_batch)
        loss.backward()
        optim.step()
        optim.zero_grad()
        loss_mean.append(loss.item())
        writer.add_scalar("Loss/train", loss, e)

    with torch.no_grad():
        mult_test = model(x_test2)
        loss_test = lossfn(mult_test, y_test2)
        loss_mean_test.append(loss_test.item())
        writer.add_scalar("Loss/test", loss_test, e)

    if (e % 10) == 0:
        print("epoch %d " % e , "train MSE: ", np.array(loss_mean).mean(), "val MSE: ", np.array(loss_mean).mean())

In [ ]:
writer = SummaryWriter(comment='sequential')
lin = torch.nn.Linear(13, 5).double()
tanh = torch.nn.Tanh().double()
lin2 = torch.nn.Linear(5, 1).double()
lossfn = torch.nn.MSELoss()
model = torch.nn.Sequential(lin, tanh, lin2)
trainloader = DataLoader(dataset=train_dataset, batch_size=20, shuffle=True)
learningRate = 0.01
epochs = 1000

optim=torch.optim.SGD(params=model.parameters(), lr=learningRate)
optim.zero_grad()
for e in tqdm(range(epochs)): 
    for x_batch, y_batch in trainloader:
        loss = lossfn(model(x_batch),y_batch)
        loss.backward()
        optim.step()
        optim.zero_grad()
        loss_mean.append(loss.item())
        writer.add_scalar("Loss/train", loss, e)

    with torch.no_grad():
        mult_test = model(x_test2)
        loss_test = lossfn(mult_test, y_test2)
        loss_mean_test.append(loss_test.item())
        writer.add_scalar("Loss/test", loss_test, e)

    if (e % 10) == 0:
        print("epoch %d " % e , "train MSE: ", np.array(loss_mean).mean(), "val MSE: ", np.array(loss_mean).mean())